# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows.
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

In [3]:
class Matrix:
    def __init__(self, n=None, m=None, data=None):
        if data:
            #Initialize with a list of lists
            self.rows = len(data)
            self.cols = len(data[0])
            for row in data:
                if len(row) != self.cols:
                    raise ValueError("All rows must have the same number of columns.")
            self.data = [list(row) for row in data]
        elif n is not None and m is not None:
            #Initialize with dimensions n x m with all zeros
            self.rows = n
            self.cols = m
            self.data = [[0] * m for _ in range(n)]
        else:
            raise ValueError("Matrix must be initialized with either dimensions or data.")

    def shape(self):
        """Returns a tuple (n, m) of the shape of the matrix."""
        return self.rows, self.cols

    def transpose(self):
        """Returns a new matrix instance that is the transpose of the matrix."""
        transposed_data = [[self.data[j][i] for j in range(self.rows)] for i in range(self.cols)]
        return Matrix(data=transposed_data)

    def row(self, n):
        """Returns the nth row as a new matrix (1 x m)."""
        if n >= self.rows or n < 0:
            raise IndexError("Row index out of range.")
        return Matrix(data=[self.data[n]])

    def column(self, n):
        """Returns the nth column as a new matrix (n x 1)."""
        if n >= self.cols or n < 0:
            raise IndexError("Column index out of range.")
        return Matrix(data=[[self.data[i][n]] for i in range(self.rows)])

    def to_list(self):
        """Returns the matrix as a list of lists."""
        return [list(row) for row in self.data]

    def block(self, n_0, n_1, m_0, m_1):
        """Returns a smaller matrix located at the n_0 to n_1 rows and m_0 to m_1 columns."""
        if not (0 <= n_0 < self.rows and 0 <= n_1 <= self.rows and 0 <= m_0 < self.cols and 0 <= m_1 <= self.cols):
            raise IndexError("Block indices are out of range.")
        block_data = [row[m_0:m_1] for row in self.data[n_0:n_1]]
        return Matrix(data=block_data)

    def __getitem__(self, indices):
        if isinstance(indices, tuple):
            i, j = indices
            if isinstance(i, slice) or isinstance(j, slice):
                return Matrix(data=[row[j] for row in self.data[i]])  # Supports slicing
            else:
                return self.data[i][j]
        else:
            return self.data[indices]

    def __setitem__(self, indices, value):
        if isinstance(indices, tuple):
            i, j = indices
            self.data[i][j] = value
        else:
            raise IndexError("Matrix indices must be provided as a tuple (i, j).")

    def __eq__(self, other):
        if not isinstance(other, Matrix):
            return False
        return self.data == other.data

    def __repr__(self):
        return "\n".join([str(row) for row in self.data])

    def assign(self, other):
        if isinstance(other, Matrix):
            if self.rows == other.rows and self.cols == other.cols:
                self.data = [list(row) for row in other.data]
            else:
                raise ValueError("Matrix dimensions must match for assignment.")
        elif isinstance(other, list):
            if len(other) == self.rows and all(len(row) == self.cols for row in other):
                self.data = [list(row) for row in other]
            else:
                raise ValueError("List dimensions must match matrix size for assignment.")
        else:
            raise TypeError("Assignment must be a Matrix or a list of lists.")




In [4]:
M1 = Matrix(3, 3)  #3x3 matrix of zeros
M2 = Matrix(data=[[1, 2, 3], [4, 5, 6], [7, 8, 9]])  # Creates a matrix from a list of lists

print("M2:")
print(M2)

# Shape of the matrix
print("Shape of M2:", M2.shape())

# Transpose of the matrix
M3 = M2.transpose()
print("Transpose of M2:")
print(M3)

# Get row and column
print("Row 1 of M2:", M2.row(1))
print("Column 2 of M2:", M2.column(2))



M2:
[1, 2, 3]
[4, 5, 6]
[7, 8, 9]
Shape of M2: (3, 3)
Transpose of M2:
[1, 4, 7]
[2, 5, 8]
[3, 6, 9]
Row 1 of M2: [4, 5, 6]
Column 2 of M2: [3]
[6]
[9]


In [5]:
print("M2 as list of lists:", M2.to_list())

# Get block
M_block = M2.block(0, 2, 0, 2)
print("Block from M2 (0 to 2, 0 to 2):")
print(M_block)

# Slicing support
print("Slice M2[:, 1:3]:")
print(M2[:, 1:3])


M2 as list of lists: [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
Block from M2 (0 to 2, 0 to 2):
[1, 2]
[4, 5]
Slice M2[:, 1:3]:
[2, 3]
[5, 6]
[8, 9]


3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

In [10]:
# Assuming the Matrix class is defined as previously provided

def constant(n, m, c):
    """Returns an n by m matrix filled with floats of value c."""
    return Matrix(data=[[float(c)] * m for _ in range(n)])

def zeros(n, m):
    """Returns an n by m matrix filled with floats of value 0."""
    return Matrix(n, m)  # Using the Matrix class to create a zero matrix

def ones(n, m):
    """Returns an n by m matrix filled with floats of value 1."""
    return Matrix(data=[[1.0] * m for _ in range(n)])

def eye(n):
    """Returns the n by n identity matrix."""
    identity_data = [[1.0 if i == j else 0.0 for j in range(n)] for i in range(n)]
    return Matrix(data=identity_data)



In [9]:
if __name__ == "__main__":
    # Create a constant matrix
    C = constant(2, 3, 5)
    print("Constant Matrix (2x3 filled with 5):")
    print(C)

    # Create a zero matrix
    Z = zeros(3, 4)
    print("\nZero Matrix (3x4):")
    print(Z)

    # Create a ones matrix
    O = ones(2, 2)
    print("\nOnes Matrix (2x2):")
    print(O)

    # Create an identity matrix
    I = eye(3)
    print("\nIdentity Matrix (3x3):")
    print(I)

Constant Matrix (2x3 filled with 5):
[5.0, 5.0, 5.0]
[5.0, 5.0, 5.0]

Zero Matrix (3x4):
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]

Ones Matrix (2x2):
[1.0, 1.0]
[1.0, 1.0]

Identity Matrix (3x3):
[1.0, 0.0, 0.0]
[0.0, 1.0, 0.0]
[0.0, 0.0, 1.0]


4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

In [12]:
class Matrix:
    def __init__(self, n=None, m=None, data=None):
        if data:
            # Initialize with a list of lists
            self.rows = len(data)
            self.cols = len(data[0])
            for row in data:
                if len(row) != self.cols:
                    raise ValueError("All rows must have the same number of columns.")
            self.data = [list(row) for row in data]
        elif n is not None and m is not None:
            # Initialize with dimensions n x m with all zeros
            self.rows = n
            self.cols = m
            self.data = [[0] * m for _ in range(n)]
        else:
            raise ValueError("Matrix must be initialized with either dimensions or data.")

    def scalarmul(self, c):
        """Returns a matrix that is the scalar product cM."""
        return Matrix(data=[[c * self.data[i][j] for j in range(self.cols)] for i in range(self.rows)])

    def add(self, other):
        """Adds two matrices M and N."""
        if not isinstance(other, Matrix):
            raise TypeError("The operand must be a Matrix.")
        if self.rows != other.rows or self.cols != other.cols:
            raise ValueError("Matrix dimensions must match for addition.")
        return Matrix(data=[[self.data[i][j] + other.data[i][j] for j in range(self.cols)] for i in range(self.rows)])

    def sub(self, other):
        """Subtracts two matrices M and N."""
        if not isinstance(other, Matrix):
            raise TypeError("The operand must be a Matrix.")
        if self.rows != other.rows or self.cols != other.cols:
            raise ValueError("Matrix dimensions must match for subtraction.")
        return Matrix(data=[[self.data[i][j] - other.data[i][j] for j in range(self.cols)] for i in range(self.rows)])

    def mat_mult(self, other):
        """Returns the matrix product of two matrices M and N."""
        if not isinstance(other, Matrix):
            raise TypeError("The operand must be a Matrix.")
        if self.cols != other.rows:
            raise ValueError("Matrix dimensions are not compatible for multiplication.")
        # Perform matrix multiplication
        product_data = [[sum(self.data[i][k] * other.data[k][j] for k in range(self.cols)) for j in range(other.cols)] for i in range(self.rows)]
        return Matrix(data=product_data)

    def element_mult(self, other):
        """Returns the element-wise product of two matrices M and N."""
        if not isinstance(other, Matrix):
            raise TypeError("The operand must be a Matrix.")
        if self.rows != other.rows or self.cols != other.cols:
            raise ValueError("Matrix dimensions must match for element-wise multiplication.")
        return Matrix(data=[[self.data[i][j] * other.data[i][j] for j in range(self.cols)] for i in range(self.rows)])

    def equals(self, other):
        """Returns true/false if M == N."""
        if not isinstance(other, Matrix):
            return False
        return self.data == other.data


    def __getitem__(self, indices):
        if isinstance(indices, tuple):
            i, j = indices
            if isinstance(i, slice) or isinstance(j, slice):
                return Matrix(data=[row[j] for row in self.data[i]])  # Supports slicing
            else:
                return self.data[i][j]
        else:
            return self.data[indices]

    def __setitem__(self, indices, value):
        if isinstance(indices, tuple):
            i, j = indices
            self.data[i][j] = value
        else:
            raise IndexError("Matrix indices must be provided as a tuple (i, j).")

    def __eq__(self, other):
        if not isinstance(other, Matrix):
            return False
        return self.data == other.data

    def __repr__(self):
        return "\n".join([str(row) for row in self.data])

    def assign(self, other):
        if isinstance(other, Matrix):
            if self.rows == other.rows and self.cols == other.cols:
                self.data = [list(row) for row in other.data]
            else:
                raise ValueError("Matrix dimensions must match for assignment.")
        elif isinstance(other, list):
            if len(other) == self.rows and all(len(row) == self.cols for row in other):
                self.data = [list(row) for row in other]
            else:
                raise ValueError("List dimensions must match matrix size for assignment.")
        else:
            raise TypeError("Assignment must be a Matrix or a list of lists.")


In [ ]:
if __name__ == "__main__":
    A = Matrix(data=[[1, 2], [3, 4]])
    B = Matrix(data=[[5, 6], [7, 8]])

    # Scalar multiplication
    C = A.scalarmul(2)
    print("Scalar Multiplication of A by 2:")
    print(C)

    # Matrix addition
    D = A.add(B)
    print("\nMatrix Addition (A + B):")
    print(D)

    # Matrix subtraction
    E = A.sub(B)
    print("\nMatrix Subtraction (A - B):")
    print(E)

    # Matrix multiplication
    F = A.mat_mult(B)
    print("\nMatrix Multiplication (A * B):")
    print(F)

    # Element-wise multiplication
    G = A.element_mult(B)
    print("\nElement-wise Multiplication (A .* B):")
    print(G)

    # Equality check
    print("\nAre A and B equal?")
    print(A.equals(B))

5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


In [16]:
class Matrix:
    def __init__(self, n=None, m=None, data=None):
        if data is not None:
            # Initialize with a list of lists
            self.rows = len(data)
            self.cols = len(data[0])
            for row in data:
                if len(row) != self.cols:
                    raise ValueError("All rows must have the same number of columns.")
            self.data = [list(row) for row in data]
        elif n is not None and m is not None:
            # Initialize with dimensions n x m with all zeros
            self.rows = n
            self.cols = m
            self.data = [[0] * m for _ in range(n)]
        else:
            raise ValueError("Matrix must be initialized with either dimensions or data.")

    def scalarmul(self, c):
        """Returns a matrix that is the scalar product cM."""
        return Matrix(data=[[c * self.data[i][j] for j in range(self.cols)] for i in range(self.rows)])

    def add(self, other):
        """Adds two matrices M and N."""
        if not isinstance(other, Matrix):
            raise TypeError("The operand must be a Matrix.")
        if self.rows != other.rows or self.cols != other.cols:
            raise ValueError("Matrix dimensions must match for addition.")
        return Matrix(data=[[self.data[i][j] + other.data[i][j] for j in range(self.cols)] for i in range(self.rows)])

    def sub(self, other):
        """Subtracts two matrices M and N."""
        if not isinstance(other, Matrix):
            raise TypeError("The operand must be a Matrix.")
        if self.rows != other.rows or self.cols != other.cols:
            raise ValueError("Matrix dimensions must match for subtraction.")
        return Matrix(data=[[self.data[i][j] - other.data[i][j] for j in range(self.cols)] for i in range(self.rows)])

    def mat_mult(self, other):
        """Returns the matrix product of two matrices M and N."""
        if not isinstance(other, Matrix):
            raise TypeError("The operand must be a Matrix.")
        if self.cols != other.rows:
            raise ValueError("Matrix dimensions are not compatible for multiplication.")
        # Perform matrix multiplication
        product_data = [[sum(self.data[i][k] * other.data[k][j] for k in range(self.cols)) for j in range(other.cols)] for i in range(self.rows)]
        return Matrix(data=product_data)

    def element_mult(self, other):
        """Returns the element-wise product of two matrices M and N."""
        if not isinstance(other, Matrix):
            raise TypeError("The operand must be a Matrix.")
        if self.rows != other.rows or self.cols != other.cols:
            raise ValueError("Matrix dimensions must match for element-wise multiplication.")
        return Matrix(data=[[self.data[i][j] * other.data[i][j] for j in range(self.cols)] for i in range(self.rows)])

    def equals(self, other):
        """Returns true/false if M == N."""
        if not isinstance(other, Matrix):
            return False
        return self.data == other.data

    # Operator overloading
    def __add__(self, other):
        return self.add(other)

    def __sub__(self, other):
        return self.sub(other)

    def __mul__(self, other):
        if isinstance(other, Matrix):
            return self.mat_mult(other)
        elif isinstance(other, (int, float)):
            return self.scalarmul(other)
        else:
            raise TypeError("Unsupported operand type(s) for *: 'Matrix' and '{}'".format(type(other).__name__))

    def __rmul__(self, other):
        """For scalar multiplication when scalar is on the left side (2 * M)."""
        return self.scalarmul(other)

    def __eq__(self, other):
        return self.equals(other)

    def __repr__(self):
        return "\n".join([str(row) for row in self.data])

    def __getitem__(self, indices):
        if isinstance(indices, tuple):
            i, j = indices
            if isinstance(i, slice) or isinstance(j, slice):
                return Matrix(data=[row[j] for row in self.data[i]])  # Supports slicing
            else:
                return self.data[i][j]
        else:
            return self.data[indices]

    def __setitem__(self, indices, value):
        if isinstance(indices, tuple):
            i, j = indices
            self.data[i][j] = value
        else:
            raise IndexError("Matrix indices must be provided as a tuple (i, j).")





In [17]:
if __name__ == "__main__":
    A = Matrix(data=[[1, 2], [3, 4]])
    B = Matrix(data=[[5, 6], [7, 8]])

    # Scalar multiplication
    C = 2 * A
    print("Scalar Multiplication of A by 2:")
    print(C)


    # Matrix addition
    D = A + B
    print("\nMatrix Addition (A + B):")
    print(D)

    # Matrix subtraction
    E = A - B
    print("\nMatrix Subtraction (A - B):")
    print(E)

    # Matrix multiplication
    F = A * B
    print("\nMatrix Multiplication (A * B):")
    print(F)

    # Equality check
    print("\nAre A and B equal?")
    print(A == B)

Scalar Multiplication of A by 2:
[2, 4]
[6, 8]

Matrix Addition (A + B):
[6, 8]
[10, 12]

Matrix Subtraction (A - B):
[-4, -4]
[-4, -4]

Matrix Multiplication (A * B):
[19, 22]
[43, 50]

Are A and B equal?
False


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [19]:
def identity_matrix(n):
    """Returns an n x n identity matrix."""
    return Matrix(data=[[1.0 if i == j else 0.0 for j in range(n)] for i in range(n)])

if __name__ == "__main__":
    # two 2x2 matrices A and B
    A = Matrix(data=[[1, 2], [3, 4]])
    B = Matrix(data=[[5, 6], [7, 8]])

    # Matrix C
    C = Matrix(data=[[9, 10], [11, 12]])

    # Associative property: (AB)C = A(BC)
    AB = A * B
    AB_C = AB * C
    BC = B * C
    A_BC = A * BC
    print("Associative Property:")
    print("(AB)C = ", AB_C)
    print("A(BC) = ", A_BC)
    print("Are they equal? ", AB_C.equals(A_BC))

    # Distributive property: A(B+C) = AB + AC
    B_plus_C = B.add(C)
    A_B_plus_C = A * B_plus_C
    AB_plus_AC = AB.add(A.scalarmul(1))  # AB + AC where AC = A * C
    AC = A * C
    AB_plus_AC = AB.add(AC)

    print("\nDistributive Property:")
    print("A(B+C) = ", A_B_plus_C)
    print("AB + AC = ", AB_plus_AC)
    print("Are they equal? ", A_B_plus_C.equals(AB_plus_AC))

    # Non-commutativity: AB ≠ BA
    BA = B * A
    print("\nNon-Commutativity:")
    print("AB = ", AB)
    print("BA = ", BA)
    print("Are they equal? ", AB.equals(BA))

    # Identity property: AI = A
    I = identity_matrix(2)
    AI = A * I
    print("\nIdentity Property:")
    print("AI = ", AI)
    print("Is AI equal to A? ", AI.equals(A))



Associative Property:
(AB)C =  [413, 454]
[937, 1030]
A(BC) =  [413, 454]
[937, 1030]
Are they equal?  True

Distributive Property:
A(B+C) =  [50, 56]
[114, 128]
AB + AC =  [50, 56]
[114, 128]
Are they equal?  True

Non-Commutativity:
AB =  [19, 22]
[43, 50]
BA =  [23, 34]
[31, 46]
Are they equal?  False

Identity Property:
AI =  [1.0, 2.0]
[3.0, 4.0]
Is AI equal to A?  True
